In [96]:
from bs4 import BeautifulSoup
import requests
import math
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import sqlite3
key_fd = open('./과제data/kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [97]:
key_fd = open('./과제data/seoulapikey.txt', mode='r')
seoul_key = key_fd.read(100)
key_fd.close()

In [98]:
seoul_url = 'http://openapi.seoul.go.kr:8088/'
url = f'{seoul_url}{seoul_key}/xml/Corona19Status/1/1000'

In [99]:
url

'http://openapi.seoul.go.kr:8088/694f57486f686a6b33387443706a41/xml/Corona19Status/1/1000'

In [100]:
result = requests.get(url)
soup = BeautifulSoup(result.text, 'xml')
soup.find('MESSAGE').get_text()

'정상 처리되었습니다'

In [101]:
total_count = int(soup.find('list_total_count').get_text())

In [102]:
total_count/1000

19.004

In [103]:
count = math.ceil(total_count/1000)
count

20

In [104]:
rows = soup.find_all('row')

In [105]:
rows[0].find('CORONA19_DATE').text

'20.12.30.'

In [106]:
rows[0].find('CORONA19_AREA').text

'기타'

In [107]:
rows[0].find('CORONA19_ID').text

'19004'

In [108]:
rows[0].find('CORONA19_CONTACT_HISTORY').text

'동부구치소 관련'

In [109]:
id_list, area_list, date_list,histroy_list = [],[],[],[]
a_url = f'{seoul_url}{seoul_key}/xml/Corona19Status'
x = 1
for i in range(count):  
    url = f'{a_url}/{x}/{x+999}'
    x += 1000
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'xml')
    rows = soup.find_all('row')
    for row in rows:
        id_list.append(row.find('CORONA19_ID').text)
        area_list.append(row.find('CORONA19_AREA').text)
        date_list.append(row.find('CORONA19_DATE').text)
        histroy_list.append(row.find('CORONA19_CONTACT_HISTORY').text)

In [110]:
df = pd.DataFrame({
    '연번' : id_list,
    '확진일' : date_list,
    '지역' : area_list,
    '접촉력' : histroy_list
})

In [111]:
len(histroy_list)

19004

In [112]:
df['지역'] = df['지역'].str.replace(' ','')

In [113]:
df['지역'][df['지역'] == '중로구']

3381    중로구
Name: 지역, dtype: object

In [114]:
df['지역'][df['지역'] == '중로구'] = '종로구'
df['지역'][df['지역'] == '앙쳔구'] = '양천구'

In [115]:
df['지역'][df['지역'] == '앙쳔구']

Series([], Name: 지역, dtype: object)

In [116]:
df['확진일'] = '20' + df['확진일']

In [117]:
df['확진일'] = df['확진일'].str.replace('.','-').str[:-1]

In [118]:
df['확진일']

0        2020-12-30
1        2020-12-30
2        2020-12-30
3        2020-12-30
4        2020-12-30
            ...    
18999     2020-1-31
19000     2020-1-30
19001     2020-1-30
19002     2020-1-30
19003     2020-1-24
Name: 확진일, Length: 19004, dtype: object

In [119]:
df['확진일'] = pd.to_datetime(df['확진일'])

In [120]:
df['확진일'] = df['확진일'].astype(str)

In [121]:
df = df.sort_values(by='확진일')

In [122]:
df.reset_index(drop=True,inplace=True)

In [123]:
df.tail()

,연번,확진일,지역,접촉력
18999,18757,2020-12-30,광진구,기타 확진자 접촉
19000,18756,2020-12-30,금천구,기타 확진자 접촉
19001,18755,2020-12-30,송파구,기타 확진자 접촉
19002,18753,2020-12-30,영등포구,영등포구 소재 의료기관
19003,19004,2020-12-30,기타,동부구치소 관련


In [124]:
conn = sqlite3.connect('./DB/covid-19.db')
cur = conn.cursor()

In [125]:
sql_table = '''
    create table '서울시 확진자 현황'(
        '연번' text not null primary key, '확진일' text not null,
        '지역' text not null, 접촉력 text not null
    )
'''

In [126]:
cur.execute(sql_table)
conn.commit()

In [127]:
sql_insert = 'insert into "서울시 확진자 현황" values(?,?,?,?)'

In [128]:
for i in df.index:
    params = list(df.loc[i])
    cur.execute(sql_insert, params)
    conn.commit()

In [129]:
conn.close()

In [130]:
conn = sqlite3.connect('./DB/covid-19.db')
cur = conn.cursor()
cur.execute(f'select * from "서울시 확진자 현황"')
rows = cur.fetchall()
conn.close()

In [131]:
df = pd.DataFrame(rows, columns=['연번','확진일','지역','접촉력'])

In [132]:
df

,연번,확진일,지역,접촉력
0,1,2020-01-24,강서구,해외 접촉
1,4,2020-01-30,마포구,해외 접촉
2,2,2020-01-30,중랑구,해외 접촉
3,3,2020-01-30,종로구,#3 접촉
4,5,2020-01-31,성북구,#5 접촉
...,...,...,...,...
18999,18757,2020-12-30,광진구,기타 확진자 접촉
19000,18756,2020-12-30,금천구,기타 확진자 접촉
19001,18755,2020-12-30,송파구,기타 확진자 접촉
19002,18753,2020-12-30,영등포구,영등포구 소재 의료기관


In [133]:
sam = df[df['지역'] == '강서구'].pivot_table('연번','접촉력',aggfunc="count")

In [134]:
sam.sort_values(by='연번', ascending=False).head(10)

,연번
접촉력,
기타 확진자 접촉,340
감염경로 조사중,222
강서구 교회 관련,164
강서구 댄스교습관련 시설,153
타시도 확진자 접촉,45
확인 중,26
8.15서울도심집회,18
강서구 소재 댄스교습시설 연관 병원,16
강서구 소재 병원관련(11월),16


In [135]:
list(sam.sort_values(by='연번', ascending=False).head(10).index)

['기타 확진자 접촉',
 '감염경로 조사중',
 '강서구 교회 관련',
 '강서구 댄스교습관련 시설',
 '타시도 확진자 접촉',
 '확인 중',
 '8.15서울도심집회',
 '강서구 소재 댄스교습시설 연관 병원',
 '강서구 소재 병원관련(11월)',
 '관악구 소재 음식점 관련']

In [136]:
sam.sort_values(by='연번', ascending=False).head(10)['연번']

접촉력
기타 확진자 접촉              340
감염경로 조사중               222
강서구 교회 관련              164
강서구 댄스교습관련 시설          153
타시도 확진자 접촉              45
확인 중                    26
8.15서울도심집회              18
강서구 소재 댄스교습시설 연관 병원     16
강서구 소재 병원관련(11월)        16
관악구 소재 음식점 관련           14
Name: 연번, dtype: int64

In [137]:
a_d = df.pivot_table('연번', '지역', aggfunc='count')

In [138]:
a_d.rename({'연번':'확진자수'},axis=1, inplace=True)

In [139]:
a_d

,확진자수
지역,
,61
강남구,936
강동구,562
강북구,429
강서구,1336
경기도광명시,1
경기도하남시,1
관악구,1017
광진구,465


In [140]:
d_day = df[df['지역'] == '강서구'].pivot_table('연번','확진일',aggfunc="count")

In [141]:
ten_day = d_day.sort_values(by='확진일', ascending=False).head(10)

In [142]:
list(ten_day.index)[::-1]


['2020-12-21',
 '2020-12-22',
 '2020-12-23',
 '2020-12-24',
 '2020-12-25',
 '2020-12-26',
 '2020-12-27',
 '2020-12-28',
 '2020-12-29',
 '2020-12-30']

'2020-12-28'

In [143]:
ten_day = d_day.sort_values(by='확진일', ascending=False).head(10)
ten_date = list(ten_day.index)
ten_val = list(ten_day['연번'].values)